In [2]:
import csv
import os
import requests as req
import json
from collections import Counter

In [3]:
#Donorschoose.org API
api_key = "DONORSCHOOSE"
url = "https://api.donorschoose.org/common/json_feed.html?max=100"

In [4]:
#Open a CSV for a list of Bay Area cities
with open("bayarea_cities.csv") as cities:
    cities_reader = csv.reader(cities)
    
    cities = [city[0] for city in cities_reader]
    
    print(len(cities))

cities = cities[69:74]
print(cities)

102
['Redwood City', 'Richmond', 'Rio Vista', 'Rohnert Park', 'Ross']


In [5]:
# Build partial query URL
query_url = url + "&APIkey=" + api_key + "&state=CA" + "&cityName="
#location_data = [req.get(query_url + city).json() for city in cities]

all_data = {}

for city in cities:
    
    results = req.get(query_url + city).json()
    all_data[city] = results   

print(json.dumps(all_data, indent=2))

{
  "Redwood City": {
    "searchTerms": "CA",
    "searchURL": "https://www.donorschoose.org/donors/search.html?APIkey=DONORSCHOOSE&cityName=Redwood+City&state=CA&utm_source=api&utm_medium=feed&utm_content=searchlink&utm_campaign=DONORSCHOOSE",
    "totalProposals": "1",
    "index": "0",
    "max": "1",
    "breadcrumb": [
      [
        "max",
        "100",
        ""
      ],
      [
        "state",
        "CA",
        "California"
      ]
    ],
    "proposals": [
      {
        "id": "2905674",
        "hashCode": "1223790304",
        "proposalURL": "https://www.donorschoose.org/project/little-scientists-big-possibilities-br/2905674/?verify=1223790304&utm_source=api&utm_medium=feed&utm_content=bodylink&utm_campaign=DONORSCHOOSE",
        "fundURL": "https://secure.donorschoose.org/donors/givingCart.html?proposalid=2905674&donationAmount=45&verify=1223790304&utm_source=api&utm_medium=feed&utm_content=fundlink&utm_campaign=DONORSCHOOSE",
        "imageURL": "https://storage.

In [6]:
city_data = {}

for cityname, data in all_data.items():
        
    #step 1: get cost to complete
    #step 2: get povertyType label
    #step 3: add this to the output vairable
    
    costcomplete = []
    povertytype = []
    
    data["proposals"]
    
    for x in data["proposals"]:
        
        costcomplete.append(float(x["costToComplete"]))
        povertytype.append(x["povertyType"]["label"])
    
    city_data[cityname] = {
        
        "costToComplete": sum(costcomplete),
        "povertyType": Counter(povertytype)
    }

city_data

{'Redwood City': {'costToComplete': 2628.35,
  'povertyType': Counter({'MODERATELOW': 1})},
 'Richmond': {'costToComplete': 39234.94,
  'povertyType': Counter({'HIGH': 22,
           'HIGHEST': 17,
           'LOW': 1,
           'MODERATEHIGH': 8,
           'MODERATELOW': 1})},
 'Rio Vista': {'costToComplete': 1047.14,
  'povertyType': Counter({'MODERATELOW': 2})},
 'Rohnert Park': {'costToComplete': 172.75,
  'povertyType': Counter({'HIGH': 1})},
 'Ross': {'costToComplete': 0, 'povertyType': Counter()}}

In [7]:
#list of cityname, costtocomplete, number of low, number of modeatelow, etc.

csv_data = []

citynames = sorted(city_data.keys())

for cityname in citynames: 
    
    data = city_data[cityname]
    
    one_row = []
    
    one_row.append(cityname)
    one_row.append(data['costToComplete'])  
    
    labeltype = ["LOW", "MODERATELOW", "MODERATE", "MODERATEHIGH", "HIGH", "HIGHEST", "UNKNOWN"]
    
    for x in labeltype:
        
        one_row.append(data['povertyType'].get(x, 0))
    
    csv_data.append(one_row)

csv_data

[['Redwood City', 2628.35, 0, 1, 0, 0, 0, 0, 0],
 ['Richmond', 39234.94, 1, 1, 0, 8, 22, 17, 0],
 ['Rio Vista', 1047.14, 0, 2, 0, 0, 0, 0, 0],
 ['Rohnert Park', 172.75, 0, 0, 0, 0, 1, 0, 0],
 ['Ross', 0, 0, 0, 0, 0, 0, 0, 0]]

In [8]:
root_path = os.path.join(os.getcwd())
data_path = os.path.join(root_path)
csvpath = os.path.join(root_path, "donorschoose_api.csv")
with open(csvpath, "a", newline="") as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=",")
    csvwriter.writerows(csv_data)